# Training DonkeyCar model with SageMaker

The **SageMaker Python SDK** makes it easy to train and deploy ML models. In this notebook we train a model from data collected from the robocar.

## Setup the environment

First we need to define a few variables that will be needed later. Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [ ]:
from sagemaker import get_execution_role

#Bucket with input data
data_location = 's3://<your S3 bucket name>'

#IAM execution role that gives SageMaker access to resources in your AWS account.
#We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

## Create the session 

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:
* The container name. This is constructed as in the shell commands above.
* The role. As defined above.
* The instance count which is the number of machines to use for training.
* The instance type which is the type of machine to use for training.
* The output path determines where the model artifact will be written.
* The session is the SageMaker session object that we defined above.
Then we use fit() on the estimator to train against the data that we uploaded above.

In [ ]:
region = sess.boto_session.region_name
image = '831212071815.dkr.ecr.{}.amazonaws.com/donkey:latest'.format(region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c5.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

### Download the model and deploy to robocar

Run the code below to get the S3 location for model data. Download this onto the robocar and test running with it.

In [ ]:
print(tree.model_data)